In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f
import warnings
warnings.filterwarnings('ignore')

spark = SparkSession.builder.master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/03 06:14:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
path_estabelecimento = '/home/rafael/Downloads/estabelecimentos/'
estabelecimento = spark.read.csv(path_estabelecimento, sep=';', inferSchema=True)


path_empresas = '/home/rafael/Downloads/socios/'
empresas = spark.read.csv(path_empresas, sep=';', inferSchema=True)

In [3]:
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

empresasColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [4]:
for index, colName in enumerate(empresasColNames):
    empresas = empresas.withColumnRenamed(f"_c{index}", colName)

In [5]:
for index, colName in enumerate(estabsColNames):
    estabelecimento = estabelecimento.withColumnRenamed(f"_c{index}", colName)

In [6]:
empresas = empresas.withColumn('qualificacao_do_socio', f.regexp_replace('qualificacao_do_socio',',','.'))
empresas = empresas.withColumn('qualificacao_do_socio', empresas['qualificacao_do_socio'].cast(DoubleType()))
empresas = empresas.withColumn("data_de_entrada_sociedade", f.to_date(empresas.data_de_entrada_sociedade.cast(StringType()),'yyyyMMdd'))

# CSV

In [7]:
empresas.write.csv(path='csv',mode='overwrite',sep=';',header=True)

In [8]:
empresas2 = spark.read.csv('csv',sep=';',inferSchema=True,header=True)

In [9]:
empresas2.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: double (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



# Parquet

In [10]:
empresas.write.parquet(path='parquet',mode='overwrite')

24/02/03 06:14:36 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/02/03 06:14:36 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/02/03 06:14:36 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/02/03 06:14:38 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/02/03 06:14:38 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [11]:
empresas_parquet = spark.read.parquet('parquet')

In [12]:
empresas_parquet.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: double (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



# Particionamento de Dados

Salva com menos partições

In [13]:
empresas.coalesce(1).write.csv(path='csv-unico',mode='overwrite',sep=';',header=True)

Salva em pastas separadas de acordo com o parttionBy, demora bastante.

In [14]:
# empresas.write.parquet(path='parquet-unico',mode='overwrite',partitionBy='cnpj_basico')

In [15]:
spark.stop()